In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import csv

In [ ]:
def parse_auto_ria(page=1):
    url = f"https://auto.ria.com/car/used/?page={page}"
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        )
    }
    resp = requests.get(url, headers=headers, timeout=10)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    data = []
    for ticket in soup.select("section.ticket-item")[:20]:
        item = {"Цена(в $)": "Na", "Пробег": "Na", "Модель_полная": "Na"}
        text = ticket.get_text(separator=" ", strip=True)

        # Цена
        price_tag = ticket.select_one("span.price-ticket") or ticket.select_one("span.green")
        if price_tag:
            item["Цена(в $)"] = price_tag.get_text(strip=True)

        # Пробег
        mileage = None
        for li in ticket.select("li"):
            li_text = li.get_text(strip=True)
            if re.search(r"[\d\s,]+км", li_text):
                mileage = re.sub(r"\s+", "", li_text).replace(",", "")
                break
        if mileage:
            item["Пробег"] = re.sub(r"[^\d]", "", mileage) + " тыс.км"

        # Модель (полная строка)
        model_tag = ticket.select_one("a.address")
        if model_tag:
            model_text = model_tag.get_text(strip=True)
            item["Модель_полная"] = model_text

        data.append(item)

    return data

# Сбор данных
all_data = []
for page in (1, 20):
    try:
        all_data += parse_auto_ria(page)
    except Exception as e:
        print(f"Ошибка на странице {page}: {e}")

# В DataFrame
df = pd.DataFrame(all_data)
df = df[(df != "Na").any(axis=1)].reset_index(drop=True)

# Разделим "Модель_полная" на "Марка" и "Модель"
df[["Марка", "Модель"]] = df["Модель_полная"].str.extract(r"^(\S+)\s+(.+)$")

# Извлекаем год из модели, если есть, и удаляем его из самой модели
df["Год"] = df["Модель"].str.extract(r"(19\d{2}|20\d{2})$")
df["Модель"] = df["Модель"].str.replace(r"(19\d{2}|20\d{2})$", "", regex=True).str.strip()

# Удаляем временную колонку
df.drop(columns=["Модель_полная"], inplace=True)

# Вывод полной таблицы
from IPython.display import display
print("Все данные:")
display(df)

# Вывод отфильтрованной таблицы без ни одного 'Na'
df_no_na = df[(df != "Na").all(axis=1)].reset_index(drop=True)
print("Только полные записи (без Na):")
display(df_no_na)

In [ ]:
# Сохранение полученных данных в data
with open("../data/data_raw.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["Цена(в $)", "Пробег", "Марка", "Модель", "Год"])
    writer.writeheader()
    writer.writerows(df)
with open("../data/data_clean.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["Цена(в $)", "Пробег", "Марка", "Модель", "Год"])
    writer.writeheader()
    writer.writerows(df_no_na)